In [76]:
import pandas as pd
import numpy as np 
import requests 

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup

In [77]:
# Create the dataframe

df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

In [78]:
# Get the wikipedia html using Beautiful Soup

html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_data,"html.parser")

In [79]:
# Search for the tables in the HTML data

tables = soup.find_all('table')

In [80]:
# Search for the specific table (easier to parse it and get only the desired table)
for index,table in enumerate(tables):
    if ("Postal Code" in str(table)):
        table_index = index
print(table_index)
tables[table_index].prettify()

0


'<table class="wikitable sortable">\n <tbody>\n  <tr>\n   <th>\n    Postal Code\n   </th>\n   <th>\n    Borough\n   </th>\n   <th>\n    Neighbourhood\n   </th>\n  </tr>\n  <tr>\n   <td>\n    M1A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n    Not assigned\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M2A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n    Not assigned\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M3A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Parkwoods\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M4A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Victoria Village\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M5A\n   </td>\n   <td>\n    Downtown Toronto\n   </td>\n   <td>\n    Regent Park, Harbourfront\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M6A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Lawrence Manor, Lawrence Heights\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M7A\n   </td>\n   <td>\n    Downtown Toronto\n   </td>\n   <td>

In [81]:
# Put the parsed HTML into the dataframe

for row in tables[table_index].tbody.find_all('tr'):
    col = row.find_all("td")
    if (col != []): 
        PostalCode =col[0].text
        Borough = col[1].text
        Neighborhood = col[2].text
        df = df.append({"PostalCode":PostalCode, "Borough":Borough,'Neighborhood':Neighborhood}, ignore_index=True)

In [82]:
df.shape

(180, 3)

In [83]:
# Get rid of the /n string at the end of the strings

df['PostalCode'] = df['PostalCode'].map(lambda x: x.rstrip('\n'))
df['Borough'] = df['Borough'].map(lambda x: x.rstrip('\n'))
df['Neighborhood'] = df['Neighborhood'].map(lambda x: x.rstrip('\n'))

In [84]:
# Ignore cells where Borough is Not assigned

df = df[df['Borough'] != "Not assigned"]
df.reset_index(inplace=True)

In [85]:
df.shape

(103, 4)

In [86]:
# Getting the latitude and longitude with geocoder library

latitude=[]
longitude=[]
for code in df['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

M3A [43.75245000000007, -79.32990999999998]
M4A [43.73057000000006, -79.31305999999995]
M5A [43.65512000000007, -79.36263999999994]
M6A [43.72327000000007, -79.45041999999995]
M7A [43.66253000000006, -79.39187999999996]
M9A [43.662630000000036, -79.52830999999998]
M1B [43.811390000000074, -79.19661999999994]
M3B [43.74923000000007, -79.36185999999998]
M4B [43.70718000000005, -79.31191999999999]
M5B [43.65739000000008, -79.37803999999994]
M6B [43.70687000000004, -79.44811999999996]
M9B [43.65034000000003, -79.55361999999997]
M1C [43.78574000000003, -79.15874999999994]
M3C [43.72168000000005, -79.34351999999996]
M4C [43.68970000000007, -79.30681999999996]
M5C [43.65215000000006, -79.37586999999996]
M6C [43.69211000000007, -79.43035999999995]
M9C [43.64857000000006, -79.57824999999997]
M1E [43.765750000000025, -79.17469999999997]
M4E [43.67709000000008, -79.29546999999997]
M5E [43.64536000000004, -79.37305999999995]
M6E [43.68784000000005, -79.45045999999996]
M1G [43.76812000000007, -79.2

In [87]:
# Appending the latitude and longitude to the dataframe
df_geocoder = df.copy()
df_geocoder['Latitude'] = latitude
df_geocoder['Longitude'] = longitude

In [88]:
df_geocoder.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.75245,-79.32991
1,3,M4A,North York,Victoria Village,43.73057,-79.31306
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [89]:
df

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [106]:
# Alternatively, merge the dataframe with the the geospatial coordinates csv

coords = pd.read_csv('Geospatial_Coordinates.csv')
coords.head()
df_merged = df.copy()
df_merged = pd.merge(left=df_merged, right=coords, left_on='PostalCode', right_on='Postal Code')
df_merged.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


Because of the unreliability of the geocoder, we decided to go forward with the version using the merged dataframes.